# **Homework 2 Phoneme Classification**

* Slides: https://docs.google.com/presentation/d/1v6HkBWiJb8WNDcJ9_-2kwVstxUWml87b9CnA16Gdoio/edit?usp=sharing
* Kaggle: https://www.kaggle.com/c/ml2022spring-hw2
* Video: TBA


In [12]:
!nvidia-smi

Thu Jun 29 14:21:10 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   49C    P5              22W / 170W |   4862MiB / 12288MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`
- `libriphone/train_labels`
- `libriphone/test_split.txt`
- `libriphone/feat/train/*.pt`: training feature<br>
- `libriphone/feat/test/*.pt`:  testing feature<br>

after running the following block.

> **Notes: if the links are dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2022spring-hw2/data) and upload it to the workspace, or you can use [the Kaggle API](https://www.kaggle.com/general/74235) to directly download the data into colab.**


### Download train/test metadata

In [13]:
# Main link
# !wget -O libriphone.zip "https://github.com/xraychen/shiny-robot/releases/download/v1.0/libriphone.zip"

# Backup Link 0
# !pip install --upgrade gdown
# !gdown --id '1o6Ag-G3qItSmYhTheX6DYiuyNzWyHyTc' --output libriphone.zip

# Backup link 1
# !pip install --upgrade gdown
# !gdown --id '1R1uQYi4QpX0tBfUWt2mbZcncdBsJkxeW' --output libriphone.zip

# Backup link 2
# !wget -O libriphone.zip "https://www.dropbox.com/s/wqww8c5dbrl2ka9/libriphone.zip?dl=1"

# Backup link 3
# !wget -O libriphone.zip "https://www.dropbox.com/s/p2ljbtb2bam13in/libriphone.zip?dl=1"

# !unzip -q libriphone.zip
# !ls libriphone

### Preparing Data

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [14]:
import os
import random
import pandas as pd
from tqdm import tqdm
import torch.backends.cudnn
import numpy as np

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):        # 确实看着连类型都没有。该函数是将所在的数组缩动位置，比如第1个就把第一行缩掉，以此类推
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n)
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def data_spit(length, x, spit=10):
    if length % (spit-1) == 0:
        num = length // spit
    else:
        num = length // (spit-1)
    x_list = [x[:num], x[num:num*2], x[num*2:num*3], x[num*3:num*4], x[num*4:num*5], x[num*5:num*6], x[num*6:num*7], x[num*7:num*8], x[num*8:num*9], x[num*9:]]
    return x_list

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=1, train_val_seed=1338):     # 有一个没有孤立的种子
    # 读取pt档的时候用0.8划分，导致实际的矩阵大小没有按照0.8划分，因为不同的pt档具备的数据量不一样多
    class_num = 41 # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'val') else 'test'
    X_list = []
    y_list = []
    label_dict = {}

    # 以下是创建了label的字典，读取完毕
    if mode != 'test':
      phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

      for line in phone_file:
          line = line.strip('\n').split(' ')
          label_dict[line[0]] = [int(p) for p in line[1:]]          
    
    # 以下直到print，是读取了train, val或者test的文件名字列表
    if split == 'train' or split == 'val':
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        # random.seed(train_val_seed)
        # random.shuffle(usage_list)      # 固定了种子的shuffle，对名单进行shuffle，导致每次划分成的train和val列表都一样
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    # 以下部分将利用名单，逐个文件进行load，然后拼成1个tensor
    max_len = 3000000                                       
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode != 'test':
      y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))       # 1个pt档中的所有数据读取后存储在feat中，第1个是[837,39]的tensor
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode != 'test':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode != 'test':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode != 'test':
        y = y[:idx]

    if mode != "test":
        # 将一个列表切割成10份，并且返回列表头
        X_list = data_spit(idx, X)
        y_list = data_spit(idx, y)

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
        print(y.shape)
        return X_list, y_list
    else:
        return X


## Define Dataset

In [15]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


## Define Model

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.block(x)
        return x


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

## Hyper-parameters

In [17]:
# data prarameters
concat_nframes = 13          # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 1               # the ratio of data used for training, the rest will be used for validation
fold_num = 10

# training parameters
seed = 1024                        # random seed
batch_size = 1024               # batch size
num_epoch = 3                   # the number of training epoch
learning_rate = 0.0001          # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved

# model parameters
input_dim = 39 * concat_nframes # the input dim of the model, you should not change the value
hidden_layers = 1               # the number of hidden layers
hidden_dim = 256                # the hidden dim

## Prepare dataset and model

In [18]:
import gc

# make some lists in advance
train_list_x = []
train_list_y = []
# val_list_x = []
# val_list_y = []

train_set = []
# val_set = []

train_loader = []
val_loader = []

# preprocess data
train_list_x, train_list_y = preprocess_data(split='train', feat_dir='./data/libriphone/libriphone/feat', phone_path='./data/libriphone/libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
# val_list_x, val_list_y = preprocess_data(split='val', feat_dir='./data/libriphone/libriphone/feat', phone_path='./data/libriphone/libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# get dataset
for i in range(fold_num):
    # get dataset
    train_set.append(LibriDataset(train_list_x[i], train_list_y[i]))
    # val_set.append(LibriDataset(val_list_x[i], val_list_y[i]))

for i in range(fold_num):
    # get dataloader
    train_fold = LibriDataset([], [])
    for j in range(fold_num):
        if j != i:
            train_fold = train_fold + train_set[j]
    val_fold = train_set[i]
    train_loader.append(DataLoader(train_fold, batch_size=batch_size, shuffle=True))
    val_loader.append(DataLoader(val_fold, batch_size=batch_size, shuffle=False))

# remove raw feature to save memory
# del train_list_x, train_list_y      #, val_list_x, val_list_y
# gc.collect()


[Dataset] - # phone classes: 41, number of utterances for train: 4286


4286it [00:05, 735.56it/s]

[INFO] train set
torch.Size([2644158, 507])
torch.Size([2644158])


In [19]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

DEVICE: cuda:0


In [20]:
import numpy as np

#fix seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [21]:
# fix random seed
same_seeds(seed)

# create model, define a loss function, and optimizer
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

## Training

In [22]:
best_acc = 0.0
for epoch in range(num_epoch):

    # training
    model.train() # set the model to training mode
    for k in range(10):
        train_acc = 0.0
        train_loss = 0.0
        val_acc = 0.0
        val_loss = 0.0

        train_acc_sum = 0
        train_loss_sum = 0
        val_acc_sum = 0
        val_loss_sum = 0

        for i, batch in enumerate(tqdm(train_loader[k])):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(features)

            loss = criterion(outputs, labels)
            loss.backward()                 # backward()是建立在cross entropy的基础上的，是该类返回对象的方法
            optimizer.step()

            _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
            train_acc += (train_pred.detach() == labels.detach()).sum().item()  # detach 似乎只是为了减少属性以便于执行相等判断
            train_loss += loss.item()

        # validation
        # if len(val_set) > 0:        # 为什么要有这个if语句？
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader[k])):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                outputs = model(features)

                loss = criterion(outputs, labels)

                _, val_pred = torch.max(outputs, 1)
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

        train_acc_sum = train_acc_sum + train_acc
        train_loss_sum = train_loss_sum + train_loss
        val_acc_sum = val_acc_sum + val_acc
        val_loss_sum = val_loss_sum + val_loss

        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc_sum/2644158*4, train_loss_sum/(len(train_loader[0])*9+2583), val_acc_sum/2644158, val_loss_sum/(len(val_loader[0])*9+1)
        ))


            # if the model improves, save a checkpoint at this epoch
    if val_acc_sum > best_acc:
        best_acc = val_acc_sum
        torch.save(model.state_dict(), model_path)
        print('saving model with acc {:.3f}'.format(best_acc/2644158))
    # else:
    #     print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
    #         epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
    #     ))

# if not validating, save the last epoch
# if len(val_set) == 0:
#     torch.save(model.state_dict(), model_path)
#     print('saving model at last epoch')


100%|██████████| 287/287 [00:01<00:00, 147.73it/s]


[001/003] Train Acc: 1.858480 Loss: 0.164311 | Val Acc: 0.063378 loss: 0.159219


100%|██████████| 287/287 [00:01<00:00, 152.78it/s]


[001/003] Train Acc: 2.114663 Loss: 0.133501 | Val Acc: 0.067401 loss: 0.144590


100%|██████████| 287/287 [00:01<00:00, 150.03it/s]


...

[003/003] Train Acc: 2.452710 Loss: 0.098003 | Val Acc: 0.076505 loss: 0.110757


100%|██████████| 287/287 [00:08<00:00, 31.94it/s]


[003/003] Train Acc: 2.456262 Loss: 0.097626 | Val Acc: 0.076485 loss: 0.110974


100%|██████████| 1/1 [00:00<00:00, 500.04it/s]

[003/003] Train Acc: 2.766102 Loss: 0.109535 | Val Acc: 0.000001 loss: 0.000014


In [23]:
del train_loader, val_loader
gc.collect()

9

In [24]:
del train_set   #, val_set
gc.collect()

0

## Testing
Create a testing dataset, and load model from the saved checkpoint.

In [25]:
# load data
test_X = preprocess_data(split='test', feat_dir='./data/libriphone/libriphone/feat', phone_path='./data/libriphone/libriphone/', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 1078


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 6084000000 bytes.

In [ ]:
# load model
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
model.load_state_dict(torch.load(model_path))

Make prediction.

In [ ]:
test_acc = 0.0
test_lengths = 0
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))